# Exploratory Data Analysis (EDA)

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

# Display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)


In [ ]:
# Load the dataset
df = pd.read_csv('../data/workforce_data.csv')

print("Dataset shape:", df.shape)
print("\nColumn names:")
print(df.columns.tolist())

# Display first few rows
print("\nFirst 5 rows:")
df.head()


In [ ]:
# Basic information about the dataset
print("Dataset Info:")
df.info()

print("\n" + "="*50)
print("Data types:")
print(df.dtypes)

print("\n" + "="*50)
print("Unique values per column:")
for col in df.columns:
    print(f"{col}: {df[col].nunique()} unique values")


In [ ]:
# Check for missing values
print("Missing values:")
missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100
missing_summary = pd.DataFrame({
    'Missing Count': missing_data,
    'Missing Percentage': missing_percent
})
print(missing_summary[missing_summary['Missing Count'] > 0])

if missing_summary['Missing Count'].sum() == 0:
    print("No missing values found!")


In [ ]:
# Statistical summary of numerical columns
print("Statistical Summary:")
numerical_cols = df.select_dtypes(include=[np.number]).columns
df[numerical_cols].describe()


In [ ]:
# Examine unique values for key categorical columns
categorical_cols = ['year', 'sector', 'subsector', 'industry_group', 'industry']

for col in categorical_cols:
    if col in df.columns:
        print(f"\n{col.upper()} - {df[col].nunique()} unique values:")
        print(df[col].value_counts().head(10))
        print("-" * 50)


In [ ]:
# Analyze gender diversity patterns
if 'percent_women' in df.columns:
    plt.figure(figsize=(15, 10))
    
    # Distribution of women percentage
    plt.subplot(2, 3, 1)
    plt.hist(df['percent_women'], bins=30, alpha=0.7, color='purple', edgecolor='black')
    plt.title('Distribution of Women Percentage')
    plt.xlabel('Percent Women')
    plt.ylabel('Frequency')
    plt.axvline(df['percent_women'].mean(), color='red', linestyle='--', 
                label=f'Mean: {df["percent_women"].mean():.1f}%')
    plt.legend()
    
    # Gender representation by sector
    plt.subplot(2, 3, 2)
    if 'sector' in df.columns:
        sector_women_avg = df.groupby('sector')['percent_women'].mean().sort_values()
        sector_women_avg.plot(kind='barh', color='lightcoral')
        plt.title('Average Women % by Sector')
        plt.xlabel('Average Percent Women')
    
    # Correlation with employment size
    plt.subplot(2, 3, 3)
    if 'total_employed_in_thousands' in df.columns:
        plt.scatter(df['total_employed_in_thousands'], df['percent_women'], alpha=0.6)
        plt.xlabel('Total Employment (thousands)')
        plt.ylabel('Percent Women')
        plt.title('Women % vs Employment Size')
        plt.xscale('log')  # Log scale for better visualization
        
        # Add correlation coefficient
        corr = df['total_employed_in_thousands'].corr(df['percent_women'])
        plt.text(0.05, 0.95, f'Correlation: {corr:.3f}', transform=plt.gca().transAxes, 
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # Time trend analysis
    plt.subplot(2, 3, 4)
    if 'year' in df.columns:
        yearly_women = df.groupby('year')['percent_women'].mean()
        yearly_women.plot(kind='line', marker='o', color='green', linewidth=2, markersize=8)
        plt.title('Average Women % Over Time')
        plt.xlabel('Year')
        plt.ylabel('Average Percent Women')
        plt.grid(True, alpha=0.3)
    
    # Gender parity analysis
    plt.subplot(2, 3, 5)
    # Define gender parity ranges
    df['gender_category'] = pd.cut(df['percent_women'], 
                                  bins=[0, 30, 40, 60, 70, 100], 
                                  labels=['Low (<30%)', 'Below Parity (30-40%)', 
                                         'Parity (40-60%)', 'Above Parity (60-70%)', 
                                         'High (>70%)'])
    gender_dist = df['gender_category'].value_counts()
    plt.pie(gender_dist.values, labels=gender_dist.index, autopct='%1.1f%%', startangle=90)
    plt.title('Gender Representation Categories')
    
    # Summary statistics
    plt.subplot(2, 3, 6)
    plt.axis('off')
    stats_text = f"""
    GENDER DIVERSITY SUMMARY
    
    Mean: {df['percent_women'].mean():.1f}%
    Median: {df['percent_women'].median():.1f}%
    Std Dev: {df['percent_women'].std():.1f}%
    Min: {df['percent_women'].min():.1f}%
    Max: {df['percent_women'].max():.1f}%
    
    Parity Range (40-60%):
    {((df['percent_women'] >= 40) & (df['percent_women'] <= 60)).sum()} records
    ({((df['percent_women'] >= 40) & (df['percent_women'] <= 60)).mean()*100:.1f}%)
    """
    plt.text(0.1, 0.5, stats_text, fontsize=9, verticalalignment='center', 
             bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8))
    
    plt.tight_layout()
    plt.show()


In [ ]:
# Analyze ethnic diversity patterns
ethnic_columns = [col for col in df.columns if 'percent_' in col and col != 'percent_women']

if ethnic_columns:
    print("Ethnic diversity columns found:")
    for col in ethnic_columns:
        print(f"- {col}")
    
    plt.figure(figsize=(16, 12))
    
    # Create subplots for each ethnic group
    n_groups = len(ethnic_columns)
    cols = min(3, n_groups)
    rows = (n_groups + cols - 1) // cols
    
    for i, col in enumerate(ethnic_columns, 1):
        plt.subplot(rows, cols, i)
        plt.hist(df[col], bins=20, alpha=0.7, edgecolor='black')
        plt.title(f'{col.replace("percent_", "").replace("_", " ").title()}')
        plt.xlabel('Percentage')
        plt.ylabel('Frequency')
        plt.axvline(df[col].mean(), color='red', linestyle='--', 
                   label=f'Mean: {df[col].mean():.1f}%')
        plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    # Summary statistics for ethnic groups
    print("\nEthnic Group Statistics:")
    ethnic_stats = df[ethnic_columns].describe()
    print(ethnic_stats)
    
    # Check ethnic composition consistency
    if len(ethnic_columns) > 1:
        df['ethnic_sum'] = df[ethnic_columns].sum(axis=1)
        print(f"\nEthnic percentages sum statistics:")
        print(df['ethnic_sum'].describe())
        print(f"Records with ethnic sum > 100%: {(df['ethnic_sum'] > 100).sum()}")
        print(f"Records with ethnic sum < 50%: {(df['ethnic_sum'] < 50).sum()}")


In [ ]:
# Question 2: Which three sectors are most gender-imbalanced and how does that relate to total workforce size?
print("QUESTION 2: Most Gender-Imbalanced Sectors")
print("=" * 50)

if 'percent_women' in df.columns and 'sector' in df.columns:
    # Calculate gender imbalance (distance from 50%)
    sector_analysis = df.groupby('sector').agg({
        'percent_women': ['mean', 'count'],
        'total_employed_in_thousands': ['sum', 'mean']
    }).round(2)
    
    # Flatten column names
    sector_analysis.columns = ['avg_women_pct', 'record_count', 'total_employment', 'avg_employment']
    
    # Calculate imbalance (distance from 50%)
    sector_analysis['gender_imbalance'] = abs(sector_analysis['avg_women_pct'] - 50)
    
    # Sort by imbalance
    most_imbalanced = sector_analysis.sort_values('gender_imbalance', ascending=False)
    
    print("Top 10 Most Gender-Imbalanced Sectors:")
    print(most_imbalanced[['avg_women_pct', 'gender_imbalance', 'total_employment']].head(10))
    
    # Visualize the relationship
    plt.figure(figsize=(12, 8))
    
    plt.subplot(2, 2, 1)
    plt.scatter(most_imbalanced['total_employment'], most_imbalanced['gender_imbalance'], 
                alpha=0.7, s=60, c='red')
    plt.xlabel('Total Employment (thousands)')
    plt.ylabel('Gender Imbalance (distance from 50%)')
    plt.title('Gender Imbalance vs Total Employment')
    plt.xscale('log')
    
    plt.subplot(2, 2, 2)
    top_3_imbalanced = most_imbalanced.head(3)
    plt.barh(range(len(top_3_imbalanced)), top_3_imbalanced['avg_women_pct'], color='coral')
    plt.yticks(range(len(top_3_imbalanced)), top_3_imbalanced.index)
    plt.xlabel('Average Women Percentage')
    plt.title('Top 3 Most Imbalanced Sectors')
    plt.axvline(50, color='black', linestyle='--', alpha=0.5, label='Gender Parity')
    plt.legend()
    
    plt.subplot(2, 2, 3)
    plt.barh(range(len(top_3_imbalanced)), top_3_imbalanced['total_employment'], color='skyblue')
    plt.yticks(range(len(top_3_imbalanced)), top_3_imbalanced.index)
    plt.xlabel('Total Employment (thousands)')
    plt.title('Employment Size of Most Imbalanced Sectors')
    plt.xscale('log')
    
    plt.subplot(2, 2, 4)
    # Show correlation
    corr = most_imbalanced['total_employment'].corr(most_imbalanced['gender_imbalance'])
    plt.text(0.5, 0.7, f'Correlation between\nEmployment Size and\nGender Imbalance:\n{corr:.3f}', 
             ha='center', va='center', transform=plt.gca().transAxes,
             bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8), fontsize=12)
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nTop 3 Most Gender-Imbalanced Sectors:")
    for i, (sector, data) in enumerate(top_3_imbalanced.head(3).iterrows(), 1):
        print(f"{i}. {sector}: {data['avg_women_pct']:.1f}% women (imbalance: {data['gender_imbalance']:.1f}%, employment: {data['total_employment']:,.0f}k)")
